In [33]:
import pandas as pd
import numpy as np
import itertools
import src.scripts as src
from multiprocessing import Pool
!pip install tqdm
from tqdm import tqdm_notebook

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [7]:
df_id = src.read_from_efs('df_id.csv')
excluded_df = src.read_from_efs('excluded_df.csv')

print('df_id shape: ', df_id.shape)
print('df shape: ', df.shape)
print('excluded_df shape: ', excluded_df.shape)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


df_id shape:  (90624128, 7)
df shape:  (90624128, 14)
excluded_df shape:  (131865, 7)


In [72]:
# extract `year_npi` pairs
excluded_year_npi_pairs = excluded_df[['year','npi']].drop_duplicates().values.tolist()
target_df = df_id[['year','npi']]

In [73]:
def get_excluded_indices(excluded_year_npi_pair):
    return df_year_npi_pairs[(target_df['year']==excluded_year_npi_pair[0]) & \
                             (target_df['npi']==excluded_year_npi_pair[1])].index.tolist()

In [74]:
print("Number of available CPU cores: ")
!cat /proc/cpuinfo | grep processor | wc -l

Number of available CPU cores: 
16


In [75]:
processors = 16
if __name__ == '__main__':
    tasks = excluded_year_npi_pairs
    function = get_excluded_indices
    with Pool(processes=processors)as p:
        results = list(tqdm_notebook(p.imap(function, tasks), total=len(tasks)))

In [ ]:
# unpack results
target_indeces = list(itertools.chain.from_iterable(results))

# insert and populate targets
target_df.insert(0, 'excluded', False)
target_df.at[target_indeces,'excluded']=True

# save to efs
src.save_to_efs(target_df, 'target_df.csv')

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,excluded,year,npi
11482,True,2013,1003042441
11483,True,2013,1003042441
11484,True,2013,1003042441
11485,True,2013,1003042441
11486,True,2013,1003042441
11487,True,2013,1003042441
11488,True,2013,1003042441
11489,True,2013,1003042441
11490,True,2013,1003042441
11491,True,2013,1003042441
